In [2]:
!pip install pandas

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import os
from sklearn import model_selection, preprocessing, linear_model, metrics, svm, naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.externals import joblib

In [4]:
CLF_PATH = 'result/clf.pkl'
TFIDF_PATH = 'result/tfidf.pkl'

In [5]:
def train_model():

    df = pd.read_csv("data/data.csv")       

    train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['text'], df['id'])

    encoder = preprocessing.LabelEncoder()

    train_y = encoder.fit_transform(train_y)
    valid_y = encoder.fit_transform(valid_y)

    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    tfidf_vect.fit(df['text'])

    xtrain_tfidf =  tfidf_vect.transform(train_x)
    xvalid_tfidf =  tfidf_vect.transform(valid_x)

#     classifier = linear_model.LogisticRegression()
    classifier = naive_bayes.MultinomialNB()

    classifier.fit(xtrain_tfidf, train_y)

    predictions = classifier.predict(xvalid_tfidf)

    accuracy = metrics.accuracy_score(predictions, valid_y)

    print(accuracy)
    
    return classifier, tfidf_vect

In [22]:
def predict(text):
    if os.path.isfile(CLF_PATH) and os.path.isfile(TFIDF_PATH):
        classifier = joblib.load(CLF_PATH)
        tfidf = joblib.load(TFIDF_PATH)
    else:
        classifier, tfidf = train_model()
        joblib.dump(classifier, CLF_PATH)
        joblib.dump(tfidf, TFIDF_PATH) 
    
    text_tfidf = tfidf.transform([text])
    
    result = classifier.predict(text_tfidf)
    
    return bool(result[0])


In [23]:
text = open("text.txt","r").read()

In [24]:
print(predict(text))

False
